In [66]:
import pandas as pd
import numpy as np
eps = np.finfo(float).eps
review = pd.read_csv('/Users/wangdong/Desktop/PyCharm WorkSpace/ML_Rearch_Practice/DecisionTree/salaries.csv')

In [67]:
featureList = []
labelList = []

In [68]:
# # Fill the label list
salary = review['salary_more_then_100k']
print(salary)
print(type(salary))
for n in salary:
    labelList.append(n)
print(labelList)

0     0
1     0
2     1
3     1
4     0
5     1
6     0
7     0
8     0
9     1
10    1
11    1
12    1
13    1
14    1
15    1
Name: salary_more_then_100k, dtype: int64
<class 'pandas.core.series.Series'>
[0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]


In [69]:
def unique_vals(dataFrame, col):
    """Find the unique values for a column in a dataFrame."""
    result = set()
    for e in dataFrame[col]:
        result.add(e)
    return result, len(result)

In [70]:
def fill_list(featureList, set):
    while(len(set) != 0):
        featureList.append(set.pop())
    return featureList

In [71]:
companySet, lenCompany = unique_vals(review, 'company')
print(companySet)
fill_list(featureList, companySet)
# while len(companySet)!= 0:
#     featureList.append(companySet.pop())

jobSet, lenJob = unique_vals(review, 'job')
degreeSet, lenDegree = unique_vals(review, 'degree')
# total_len for featureList
totalLen = lenCompany + lenDegree + lenJob

fill_list(featureList, jobSet)
fill_list(featureList, degreeSet)
print(totalLen)
print("featureList ")
print(featureList)
featureList_n = []

{'google', 'abc pharma', 'facebook'}
8
featureList 
['google', 'abc pharma', 'facebook', 'computer programmer', 'sales executive', 'business manager', 'bachelors', 'masters']


In [72]:
for i in range(0, len(review.index)):
    tempArray = []
    temp = review.iloc[i]
    temp = set(temp)
    for n in range(0, totalLen):
        if(temp.__contains__(featureList[n])):
            tempArray.append(1)
        else:
            tempArray.append(0)
    featureList_n.append(tempArray)
print(featureList)
print(featureList_n)

['google', 'abc pharma', 'facebook', 'computer programmer', 'sales executive', 'business manager', 'bachelors', 'masters']
[[1, 0, 0, 0, 1, 0, 1, 0], [1, 0, 0, 0, 1, 0, 0, 1], [1, 0, 0, 0, 0, 1, 1, 0], [1, 0, 0, 0, 0, 1, 0, 1], [1, 0, 0, 1, 0, 0, 1, 0], [1, 0, 0, 1, 0, 0, 0, 1], [0, 1, 0, 0, 1, 0, 0, 1], [0, 1, 0, 1, 0, 0, 1, 0], [0, 1, 0, 0, 0, 1, 1, 0], [0, 1, 0, 0, 0, 1, 0, 1], [0, 0, 1, 0, 1, 0, 1, 0], [0, 0, 1, 0, 1, 0, 0, 1], [0, 0, 1, 0, 0, 1, 1, 0], [0, 0, 1, 0, 0, 1, 0, 1], [0, 0, 1, 1, 0, 0, 1, 0], [0, 0, 1, 1, 0, 0, 0, 1]]


In [73]:
def find_entropy(df):
    # To make the code generic, changing target variable class name
    labelColumn = df.keys()[-1]
    entropy = 0
    # Create a hash list for elements in labelColumn
    values = df[labelColumn].unique()
    for value in values:
        # fraction = the number value in label / the total number of elements in column
        fraction = df[labelColumn].value_counts()[value] / len(df[labelColumn])
        # entropy equation
        entropy += -fraction * np.log2(fraction)
    return entropy

In [74]:
def find_entropy_attribute(df, attribute):

    # We can choose the column by changing the df.keys[number]
    labelColumn = df.keys()[-1]
    label_variables = df[labelColumn].unique()
    # This gives different features in that attribute
    feature_Variables = df[attribute].unique()
    entropy2 = 0
    for fv in feature_Variables:
        entropy = 0
        # Here it must be O(n^2)
        for lv in label_variables:
            # Find the intersection of certain feature value and certain label value
            # numerator
            num = len(df[attribute][df[attribute] == fv][df[labelColumn] == lv])
            # denominator
            den = len(df[attribute][df[attribute] == fv])
            # Here we use eps as buffer to make sure the faction is float type
            fraction = num / (den + eps)
            entropy += -fraction * np.log2(fraction + eps)
        # fraction2 is the one about the certain feature variable / total elements in a column
        fraction2 = den / len(df)
        # get the sum result
        entropy2 += -fraction2 * entropy
    return abs(entropy2)

In [75]:
# The method is to find the node(column) that has highest information gain
def find_winner(df):
    # the entropy of list that temporarily saves the values of each entropy
    Entropy_att = []
    # the information gain of each feature based on label entropy
    IG = []
    # Here we iterate the column one by one except the label column.
    for key in df.keys()[:-1]:
        # Find the entropy of certain feature
        Entropy_att.append(find_entropy_attribute(df, key))
        # Find the information gain for certain feature
        IG.append(find_entropy(df) - find_entropy_attribute(df, key))
    # Among the all features, will choose the one that has highest
    # In this method, we return the column
    return df.keys()[:-1][np.argmax(IG)]

In [76]:
def get_subtable(df, node, value):
    # drop the value used already
    return df[df[node] == value].reset_index(drop=True)

In [77]:
def buildTree(df, tree=None):
    labelColumn = df.keys()[-1]

    # Get attribute with maximum information gain
    node = find_winner(df)

    # Get hash value of that attribute(node)
    attValue = np.unique(df[node])

    # Initialize an empty dictionary to create tree
    # If tree is not none, the skip it.
    if tree is None:
        tree = {}
        tree[node] = {}

    # We make loop to construct a tree by calling this function recursively.
    # In this we check if the subset is pure and stops if it is pure.
    for value in attValue:
        subtable = get_subtable(df, node, value)
        clValue, counts = np.unique(subtable['salary_more_than_100k'], return_counts=True)

        if len(counts) == 1:  # Checking purity of subset
            # Here value is the key, the first(only) line of subtable is the leaf element
            tree[node][value] = clValue[0]
            #
        else:
            # Recursively produce a new tree{} and put into the one of hash value of on feature
            # That is method to create the subtree
            tree[node][value] = buildTree(subtable)  # Calling the function recursively
    # Here we return a data type of tree{{...}}
    return tree

In [78]:
#Tester
header_list = featureList
header_list.append('salary_more_than_100k')
df_array = featureList_n
for i in range(0, len(labelList)):
    df_array[i].append(labelList[i])
print(df_array)

df = pd.DataFrame(df_array, columns=header_list)
print(df)



[[1, 0, 0, 0, 1, 0, 1, 0, 0], [1, 0, 0, 0, 1, 0, 0, 1, 0], [1, 0, 0, 0, 0, 1, 1, 0, 1], [1, 0, 0, 0, 0, 1, 0, 1, 1], [1, 0, 0, 1, 0, 0, 1, 0, 0], [1, 0, 0, 1, 0, 0, 0, 1, 1], [0, 1, 0, 0, 1, 0, 0, 1, 0], [0, 1, 0, 1, 0, 0, 1, 0, 0], [0, 1, 0, 0, 0, 1, 1, 0, 0], [0, 1, 0, 0, 0, 1, 0, 1, 1], [0, 0, 1, 0, 1, 0, 1, 0, 1], [0, 0, 1, 0, 1, 0, 0, 1, 1], [0, 0, 1, 0, 0, 1, 1, 0, 1], [0, 0, 1, 0, 0, 1, 0, 1, 1], [0, 0, 1, 1, 0, 0, 1, 0, 1], [0, 0, 1, 1, 0, 0, 0, 1, 1]]
    google  abc pharma  facebook  computer programmer  sales executive  \
0        1           0         0                    0                1   
1        1           0         0                    0                1   
2        1           0         0                    0                0   
3        1           0         0                    0                0   
4        1           0         0                    1                0   
5        1           0         0                    1                0   
6        0       

In [79]:
tree = buildTree(df)
import pprint
pprint.pprint(tree)

{'facebook': {0: {'sales executive': {0: {'bachelors': {0: 1,
                                                        1: {'google': {0: 0,
                                                                       1: {'computer programmer': {0: 1,
                                                                                                   1: 0}}}}}},
                                      1: 0}},
              1: 1}}
